# Convolution Neural Network - CNN Baseline

Convolution Neural Networks or CNN or Convnets are the current state of the art for most computer vision tasks.

This notebook will aply k-fold and use the concept of transfer learning.

Transfer learning:
- Train a CNN using 6 emotions dataset known as FER2013 dataset
- Strip out the top layers of that trained CNN architecture
- Freeze the weights of that architecture in hopes it learned high level features already
- Add back other layers that are trainable
- Train using our Smile dataset to learn specific features

In [3]:
# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Plot architecture
import keras
import pydot
from keras.utils.vis_utils import model_to_dot

from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

import os
from PIL import Image

Using TensorFlow backend.


# Data  
  
The first step is to import images and store their **pixel** values in a dataframe. This will allow us to create 10 fold by using the index of the rows.  
  
We will also **shuffle** the dataframe because the first 200 images are smilling and next 200 images are neutral.

In [4]:
# Read the annotations file that contains the label and the image file name
labels = pd.read_csv('./SMILE_Dataset/annotations.csv', header=None, names=['fname','label'])

# Shuffle data
labels = labels.sample(frac=1).reset_index()

# Use a list comprehension to loop over image file names and import one by one and store pixel values
x = np.array([image.img_to_array(image.load_img('./SMILE_Dataset/all/'+fname, target_size=(128, 128))) for fname in labels['fname']])

# Because the names are strings, the neural network only takes in numerical formats so we will one-hot encode the label
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['label'])
y = integer_encoded

Now we have two variables.  
  
x: all of the values for our images  
y: all of the labels (0:1)
  
Now we have finished working with the data. Let's define an architecture for our CNN.

# Models

In [30]:
def pizza_model():
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Modified size to 5
    model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

In [5]:
def build_model2():
    
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5,5),padding = 'Same', activation ='relu', input_shape=(128, 128, 3)))
    model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0),
              metrics=['acc'])
    return model

# K-Fold

In [6]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []

# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
# In this case, 50-50
kf = StratifiedKFold(n_splits=10)

# Just for printing purposes
id = 1

for train_index, test_index in kf.split(x,y):
    print('Kfold iteration {}/10'.format(id))
    print('Total images: {} ---- Train images: {} ---- Test images: {}'.format(len(x),len(train_index),len(test_index)))

    id += 1 
    
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    model = build_model2()
    
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.1, 
                                 shear_range=0.1,
                                 zoom_range=0.1)   
    
    datagen.fit(X_train)

    # Secret sauce to get 3-5 % accuracy more
    # Adjust the learning rate over time. (Like we saw in class!)
    # The learning rate determines the size of the steps taken during the gradient descent process.
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    
    # Used to prevent overfitting. 
    # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 20), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 20,
                              callbacks=[learning_rate_reduction])

    
    y_pred = model.predict(X_test)
    y_pred = [np.round(p[0]) for p in y_pred]
    
    print(classification_report(y_test, y_pred))
    reports.append(classification_report(y_test, y_pred,output_dict=True))
    


Kfold iteration 1/10
Total images: 400 ---- Train images: 360 ---- Test images: 40

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
18/18 [==============================] - 49s 3s/step - loss: 1.7975 - acc: 0.5583 - val_loss: 0.8423 - val_acc: 0.8000
Epoch 2/20
18/18 [==============================] - 46s 3s/step - loss: 0.6023 - acc: 0.6389 - val_loss: 13.1953 - val_acc: 0.8750
Epoch 3/20
18/18 [==============================] - 46s 3s/step - loss: 0.8504 - acc: 0.7139 - val_loss: 116.2637 - val_acc: 0.5000
Epoch 4/20
18/18 [==============================] - 46s 3s/step - loss: 0.4712 - acc: 0.8278 - val_loss: 206.5266 - val_acc: 0.5000
Epoch 5/20
11/18 [=================>............] - ETA: 17s - loss: 0.3798 - acc: 0.8500

KeyboardInterrupt: 

In [32]:
# We loop over all reports (1 per fold) and then compute the average of all weighted f1 scores
final_f1_score = np.mean([rep['weighted avg']['f1-score'] for rep in reports])

print('Final F1-Score is: {}%'.format(np.round(final_f1_score*100,2)))

Final F1-Score is: 96.21%
